In [1]:
import numpy as np
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
def compute_ndvi(red, nir, nodata):
    ndvi = (nir - red) / (nir + red)
    ndvi *= 1000.0
    ndvi[ndvi < -1000] = nodata
    ndvi[ndvi > 1000] = nodata
    ndvi[~np.isfinite(ndvi)] = nodata
    return ndvi.astype(np.int32)
    

In [ ]:
def compute_ndwi(green, nir, nodata):
    ndwi = (green - nir) / (green + nir)
    ndwi *= 1000.0
    ndwi[ndwi < -1000] = nodata
    ndwi[ndwi > 1000] = nodata
    ndwi[~np.isfinite(ndwi)] = nodata
    return ndwi.astype(np.int32)
    

In [ ]:
geotiff_path = "C:\\Users\\Sharath Chandra\\Downloads\\20231017_110049_PNEO-03_1_1_30cm_RD_12bit_RGBNED_DeLier\\20231017_110049_PNEO-03_1_1_30cm_RD_12bit_RGBNED_DeLier.tif"
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio(geotiff_path, chunks={"x": 4096, "y": 4096, "band": 1})

In [ ]:
geotiff_da

In [ ]:
geotiff_da.band

In [ ]:
geotiff_da.long_name

In [ ]:
geotiff_da.isel(band=0).indexes

In [ ]:
geotiff_da.isel(band=0)

In [ ]:
green_band = geotiff_da.isel(band=1)  # green band
nir_band = geotiff_da.isel(band=3)  # nir band

# sub = (green_band - nir_band)

In [ ]:
plt.imshow(green_band, cmap='gray')  # Use 'gray' colormap for single-band images
plt.title(f"Band {1}")
plt.colorbar()
plt.show()


In [ ]:
ndwi = xr.apply_ufunc(
    compute_ndwi,
    green_band, nir_band, 
    kwargs={"nodata": geotiff_da.rio.nodata},
    dask="parallelized", output_dtypes=[np.int32]
)

In [ ]:
print(ndwi.min().values)
print(ndwi.max().values)


In [ ]:
plt.imshow(ndwi, cmap='RdBu')  # Use 'gray' colormap for single-band images
plt.title(f"NDWI")
plt.colorbar()
plt.show()
